In [29]:
from __future__ import division

import os
import itertools
import math
import numpy as np
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder

from learning2learn.wrangle import (synthesize_data, 
                                    get_train_test_parameters,
                                    build_train_set_bits,
                                    build_test_trials_o1_bits, 
                                    build_test_trials_o2_bits)
from learning2learn.util import (train_test_split, add_noise, 
                                 train_model, evaluate_generalization)
from learning2learn.models import build_model

In [2]:
nb_categories = 10
nb_exemplars = 10
nb_test_trials = 2000

In [3]:
np.random.seed(0)
(shape_set_train, shape_set_test), \
(color_set_train, color_set_test), \
(texture_set_train, texture_set_test) = \
        get_train_test_parameters(images=False, nb_bits=20)
df_train, labels = synthesize_data(nb_categories, nb_exemplars)
ohe = OneHotEncoder(sparse=False)
Y_train = ohe.fit_transform(labels.reshape(-1, 1))

In [16]:
ohe = OneHotEncoder(sparse=False)
Y_train = ohe.fit_transform(labels.reshape(-1, 1))

/Users/reubenfeinman/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [4]:
X_train = build_train_set_bits(df_train, shape_set_train, color_set_train, texture_set_train)

In [5]:
X_test_order1 = build_test_trials_o1_bits(
    df_train, shape_set_train, shape_set_test, color_set_train,
    color_set_test, texture_set_train, texture_set_test,
    nb_trials=nb_test_trials
)

In [6]:
X_test_order2 = build_test_trials_o2_bits(
    shape_set_test, color_set_test, texture_set_test,
    nb_trials=nb_test_trials
)

In [48]:
def compute_diff(x, x_p):
    diff = np.abs(x.flatten() - x_p.flatten())
    
    return np.sum(diff)/len(diff)

In [49]:
p=0.
X_train_p = add_noise(X_train, p=p)
X_test_order1_p = add_noise(X_test_order1, p=p)
X_test_order2_p = add_noise(X_test_order2, p=p)
print(compute_diff(X_train_p, X_train))
print(compute_diff(X_test_order1_p, X_test_order1))
print(compute_diff(X_test_order2_p, X_test_order2))

0.0
0.0
0.0


In [43]:
from keras.layers.core import Dense, Activation, Dropout
from keras.regularizers import l2

def mlp0(nb_in, nb_classes):
    layers = [
        #Dense(30, input_dim=nb_in, kernel_regularizer=l2(0.01)),
        Dense(30, input_dim=nb_in),
        Activation('relu'),
        #Dense(nb_classes, kernel_regularizer=l2(0.01)),
        Dense(nb_classes),
        Activation('softmax')
    ]

    return build_model(layers)

def mlp1(nb_in, nb_classes):
    layers = [
        Dense(30, input_dim=nb_in, kernel_regularizer=l2(0.01)),
        Activation('relu'),
        Dense(nb_classes),
        Activation('softmax')
    ]

    return build_model(layers)

In [25]:
def model_trial(
    model_fn, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_runs=5, nb_epoch=100, burn_period=20, layer_num=-3
):
    scores_o1 = np.zeros(nb_runs)
    scores_o2 = np.zeros(nb_runs)
    for i in range(nb_runs):
        print('Trial #%i' % (i+1))
        model = model_fn(
            nb_in=X_train.shape[-1], 
            nb_classes=Y_train.shape[-1]
        )
        weights_file = '../data/temp.h5'
        if os.path.isfile(weights_file):
            os.remove(weights_file)
        checkpoint = ModelCheckpoint(
            weights_file,
            monitor='loss',
            save_best_only=True,
            save_weights_only=True,
            period=2
        )
        train_model(
            model, X_train, Y_train, epochs=nb_epoch,
            validation_data=None, batch_size=32,
            checkpoint=checkpoint, burn_period=burn_period
        )
        model.load_weights(weights_file)
        score_o1 = evaluate_generalization(
            model, X_test_o1, layer_num=layer_num,
            batch_size=128
        )
        score_o2 = evaluate_generalization(
            model, X_test_o2, layer_num=layer_num,
            batch_size=128
        )
        print('\nScore O1: %0.4f' % score_o1)
        print('Score O2: %0.4f\n' % score_o2)
        scores_o1[i] = score_o1
        scores_o2[i] = score_o2
    
    print('\nScore O1 avg: %0.4f' % scores_o1.mean())
    print('Score O2 avg: %0.4f' % scores_o2.mean())
    return scores_o1, scores_o2

In [26]:
# model = mlp0(
#     nb_in=X_train.shape[-1],
#     nb_classes=Y_train.shape[-1]
# )

In [ ]:
# Model 0, no noise

In [38]:
scores_o1, scores_o2 = model_trial(
    mlp0, X_train_p, Y_train, X_test_order1_p, X_test_order2_p, 
    nb_runs=10, nb_epoch=100, burn_period=20
)

Trial #1
Epoch 1/20
100/100 [==============================] - 1s 7ms/step - loss: 2.3050 - acc: 0.1800
Epoch 2/20
100/100 [==============================] - 0s 56us/step - loss: 2.1809 - acc: 0.2500
Epoch 3/20
100/100 [==============================] - 0s 74us/step - loss: 2.1062 - acc: 0.2900
Epoch 4/20
100/100 [==============================] - 0s 71us/step - loss: 2.0559 - acc: 0.3200
Epoch 5/20
100/100 [==============================] - 0s 61us/step - loss: 2.0177 - acc: 0.3200
Epoch 6/20
100/100 [==============================] - 0s 77us/step - loss: 1.9818 - acc: 0.3500
Epoch 7/20
100/100 [==============================] - 0s 63us/step - loss: 1.9429 - acc: 0.3900
Epoch 8/20
100/100 [==============================] - 0s 72us/step - loss: 1.9039 - acc: 0.4200
Epoch 9/20
100/100 [==============================] - 0s 60us/step - loss: 1.8672 - acc: 0.4500
Epoch 10/20
100/100 [==============================] - 0s 81us/step - loss: 1.8265 - acc: 0.5100
Epoch 11/20
100/100 [==========

100/100 [==============================] - 0s 118us/step - loss: 0.1025 - acc: 1.0000
Epoch 65/80
100/100 [==============================] - 0s 66us/step - loss: 0.0973 - acc: 1.0000
Epoch 66/80
100/100 [==============================] - 0s 127us/step - loss: 0.0929 - acc: 1.0000
Epoch 67/80
100/100 [==============================] - 0s 55us/step - loss: 0.0883 - acc: 1.0000
Epoch 68/80
100/100 [==============================] - 0s 131us/step - loss: 0.0838 - acc: 1.0000
Epoch 69/80
100/100 [==============================] - 0s 65us/step - loss: 0.0803 - acc: 1.0000
Epoch 70/80
100/100 [==============================] - 0s 112us/step - loss: 0.0760 - acc: 1.0000
Epoch 71/80
100/100 [==============================] - 0s 65us/step - loss: 0.0718 - acc: 1.0000
Epoch 72/80
100/100 [==============================] - 0s 111us/step - loss: 0.0678 - acc: 1.0000
Epoch 73/80
100/100 [==============================] - 0s 57us/step - loss: 0.0636 - acc: 1.0000
Epoch 74/80
100/100 [================

Epoch 48/80
100/100 [==============================] - 0s 137us/step - loss: 0.4416 - acc: 0.9900
Epoch 49/80
100/100 [==============================] - 0s 57us/step - loss: 0.4264 - acc: 0.9900
Epoch 50/80
100/100 [==============================] - 0s 119us/step - loss: 0.4072 - acc: 1.0000
Epoch 51/80
100/100 [==============================] - 0s 61us/step - loss: 0.3927 - acc: 1.0000
Epoch 52/80
100/100 [==============================] - 0s 144us/step - loss: 0.3741 - acc: 1.0000
Epoch 53/80
100/100 [==============================] - 0s 53us/step - loss: 0.3593 - acc: 1.0000
Epoch 54/80
100/100 [==============================] - 0s 130us/step - loss: 0.3421 - acc: 1.0000
Epoch 55/80
100/100 [==============================] - 0s 65us/step - loss: 0.3302 - acc: 1.0000
Epoch 56/80
100/100 [==============================] - 0s 136us/step - loss: 0.3118 - acc: 1.0000
Epoch 57/80
100/100 [==============================] - 0s 68us/step - loss: 0.2992 - acc: 1.0000
Epoch 58/80
100/100 [====

100/100 [==============================] - 0s 57us/step - loss: 0.5829 - acc: 1.0000
Epoch 32/80
100/100 [==============================] - 0s 122us/step - loss: 0.5582 - acc: 1.0000
Epoch 33/80
100/100 [==============================] - 0s 59us/step - loss: 0.5337 - acc: 1.0000
Epoch 34/80
100/100 [==============================] - 0s 118us/step - loss: 0.5123 - acc: 1.0000
Epoch 35/80
100/100 [==============================] - 0s 61us/step - loss: 0.4855 - acc: 1.0000
Epoch 36/80
100/100 [==============================] - 0s 120us/step - loss: 0.4649 - acc: 1.0000
Epoch 37/80
100/100 [==============================] - 0s 61us/step - loss: 0.4426 - acc: 1.0000
Epoch 38/80
100/100 [==============================] - 0s 109us/step - loss: 0.4245 - acc: 1.0000
Epoch 39/80
100/100 [==============================] - 0s 72us/step - loss: 0.4073 - acc: 1.0000
Epoch 40/80
100/100 [==============================] - 0s 120us/step - loss: 0.3876 - acc: 1.0000
Epoch 41/80
100/100 [================

100/100 [==============================] - 0s 121us/step - loss: 1.0105 - acc: 0.8800
Epoch 15/80
100/100 [==============================] - 0s 54us/step - loss: 0.9750 - acc: 0.9000
Epoch 16/80
100/100 [==============================] - 0s 119us/step - loss: 0.9397 - acc: 0.9100
Epoch 17/80
100/100 [==============================] - 0s 61us/step - loss: 0.9083 - acc: 0.8900
Epoch 18/80
100/100 [==============================] - 0s 120us/step - loss: 0.8802 - acc: 0.9000
Epoch 19/80
100/100 [==============================] - 0s 59us/step - loss: 0.8446 - acc: 0.9100
Epoch 20/80
100/100 [==============================] - 0s 108us/step - loss: 0.8144 - acc: 0.9300
Epoch 21/80
100/100 [==============================] - 0s 66us/step - loss: 0.7869 - acc: 0.9300
Epoch 22/80
100/100 [==============================] - ETA: 0s - loss: 0.8134 - acc: 0.937 - 0s 112us/step - loss: 0.7591 - acc: 0.9500
Epoch 23/80
100/100 [==============================] - 0s 61us/step - loss: 0.7352 - acc: 0.9500

...

100/100 [==============================] - 0s 59us/step - loss: 0.2179 - acc: 1.0000
Epoch 48/80
100/100 [==============================] - 0s 121us/step - loss: 0.2078 - acc: 1.0000
Epoch 49/80
100/100 [==============================] - 0s 59us/step - loss: 0.1981 - acc: 1.0000
Epoch 50/80
100/100 [==============================] - 0s 119us/step - loss: 0.1872 - acc: 1.0000
Epoch 51/80
100/100 [==============================] - 0s 62us/step - loss: 0.1763 - acc: 1.0000
Epoch 52/80
100/100 [==============================] - 0s 116us/step - loss: 0.1679 - acc: 1.0000
Epoch 53/80
100/100 [==============================] - 0s 65us/step - loss: 0.1598 - acc: 1.0000
Epoch 54/80
100/100 [==============================] - 0s 115us/step - loss: 0.1515 - acc: 1.0000
Epoch 55/80
100/100 [==============================] - 0s 56us/step - loss: 0.1435 - acc: 1.0000
Epoch 56/80
100/100 [==============================] - 0s 119us/step - loss: 0.1345 - acc: 1.0000
Epoch 57/80
100/100 [================

100/100 [==============================] - 0s 64us/step - loss: 0.5732 - acc: 0.9900
Epoch 32/80
100/100 [==============================] - 0s 119us/step - loss: 0.5479 - acc: 0.9900
Epoch 33/80
100/100 [==============================] - 0s 69us/step - loss: 0.5238 - acc: 0.9900
Epoch 34/80
100/100 [==============================] - 0s 123us/step - loss: 0.5015 - acc: 0.9900
Epoch 35/80
100/100 [==============================] - 0s 68us/step - loss: 0.4811 - acc: 0.9900
Epoch 36/80
100/100 [==============================] - 0s 121us/step - loss: 0.4596 - acc: 0.9900
Epoch 37/80
100/100 [==============================] - 0s 74us/step - loss: 0.4419 - acc: 0.9900
Epoch 38/80
100/100 [==============================] - 0s 117us/step - loss: 0.4223 - acc: 0.9900
Epoch 39/80
100/100 [==============================] - 0s 62us/step - loss: 0.4010 - acc: 0.9900
Epoch 40/80
100/100 [==============================] - 0s 113us/step - loss: 0.3824 - acc: 0.9900
Epoch 41/80
100/100 [================

100/100 [==============================] - 0s 56us/step - loss: 1.0496 - acc: 0.9300
Epoch 16/80
100/100 [==============================] - 0s 118us/step - loss: 1.0161 - acc: 0.9600
Epoch 17/80
100/100 [==============================] - 0s 63us/step - loss: 0.9816 - acc: 0.9500
Epoch 18/80
100/100 [==============================] - 0s 119us/step - loss: 0.9524 - acc: 0.9500
Epoch 19/80
100/100 [==============================] - 0s 51us/step - loss: 0.9207 - acc: 0.9500
Epoch 20/80
100/100 [==============================] - 0s 117us/step - loss: 0.8922 - acc: 0.9600
Epoch 21/80
100/100 [==============================] - 0s 62us/step - loss: 0.8603 - acc: 0.9700
Epoch 22/80
100/100 [==============================] - 0s 112us/step - loss: 0.8288 - acc: 0.9700
Epoch 23/80
100/100 [==============================] - 0s 60us/step - loss: 0.8045 - acc: 0.9600
Epoch 24/80
100/100 [==============================] - 0s 125us/step - loss: 0.7719 - acc: 0.9900
Epoch 25/80
100/100 [================

100/100 [==============================] - 0s 54us/step - loss: 1.7025 - acc: 0.5100
Epoch 18/20
100/100 [==============================] - 0s 69us/step - loss: 1.6619 - acc: 0.5200
Epoch 19/20
100/100 [==============================] - 0s 54us/step - loss: 1.6277 - acc: 0.5500
Epoch 20/20
100/100 [==============================] - 0s 52us/step - loss: 1.5914 - acc: 0.5900
Epoch 1/80
100/100 [==============================] - 0s 71us/step - loss: 1.5502 - acc: 0.6400
Epoch 2/80
100/100 [==============================] - 0s 1ms/step - loss: 1.5130 - acc: 0.6300
Epoch 3/80
100/100 [==============================] - 0s 59us/step - loss: 1.4722 - acc: 0.7000
Epoch 4/80
100/100 [==============================] - 0s 129us/step - loss: 1.4360 - acc: 0.6800
Epoch 5/80
100/100 [==============================] - 0s 58us/step - loss: 1.3990 - acc: 0.7600
Epoch 6/80
100/100 [==============================] - 0s 119us/step - loss: 1.3600 - acc: 0.7400
Epoch 7/80
100/100 [===========================

100/100 [==============================] - 0s 125us/step - loss: 0.0494 - acc: 1.0000

Score O1: 0.9735
Score O2: 0.8820


Score O1 avg: 0.9561
Score O2 avg: 0.8645


In [39]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.9561
Score O2 avg: 0.8645


In [ ]:
# Model 1, no noise

In [44]:
scores_o1, scores_o2 = model_trial(
    mlp1, X_train_p, Y_train, X_test_order1_p, X_test_order2_p, 
    nb_runs=10, nb_epoch=100, burn_period=20
)

Trial #1
Epoch 1/20
100/100 [==============================] - 1s 8ms/step - loss: 2.9788 - acc: 0.0600
Epoch 2/20
100/100 [==============================] - 0s 64us/step - loss: 2.8331 - acc: 0.0700
Epoch 3/20
100/100 [==============================] - 0s 66us/step - loss: 2.7516 - acc: 0.1000
Epoch 4/20
100/100 [==============================] - 0s 65us/step - loss: 2.6863 - acc: 0.1200
Epoch 5/20
100/100 [==============================] - 0s 65us/step - loss: 2.6274 - acc: 0.1400
Epoch 6/20
100/100 [==============================] - 0s 63us/step - loss: 2.5828 - acc: 0.1900
Epoch 7/20
100/100 [==============================] - 0s 66us/step - loss: 2.5389 - acc: 0.2200
Epoch 8/20
100/100 [==============================] - 0s 64us/step - loss: 2.4947 - acc: 0.2100
Epoch 9/20
100/100 [==============================] - 0s 62us/step - loss: 2.4540 - acc: 0.2900
Epoch 10/20
100/100 [==============================] - 0s 68us/step - loss: 2.4129 - acc: 0.3000
Epoch 11/20
100/100 [==========

100/100 [==============================] - 0s 55us/step - loss: 0.4911 - acc: 1.0000
Epoch 66/80
100/100 [==============================] - 0s 131us/step - loss: 0.4832 - acc: 1.0000
Epoch 67/80
100/100 [==============================] - 0s 68us/step - loss: 0.4753 - acc: 1.0000
Epoch 68/80
100/100 [==============================] - 0s 142us/step - loss: 0.4690 - acc: 1.0000
Epoch 69/80
100/100 [==============================] - 0s 56us/step - loss: 0.4625 - acc: 1.0000
Epoch 70/80
100/100 [==============================] - 0s 129us/step - loss: 0.4544 - acc: 1.0000
Epoch 71/80
100/100 [==============================] - 0s 65us/step - loss: 0.4486 - acc: 1.0000
Epoch 72/80
100/100 [==============================] - 0s 124us/step - loss: 0.4421 - acc: 1.0000
Epoch 73/80
100/100 [==============================] - 0s 73us/step - loss: 0.4368 - acc: 1.0000
Epoch 74/80
100/100 [==============================] - 0s 132us/step - loss: 0.4314 - acc: 1.0000
Epoch 75/80
100/100 [================

Epoch 49/80
100/100 [==============================] - 0s 76us/step - loss: 0.5842 - acc: 1.0000
Epoch 50/80
100/100 [==============================] - 0s 123us/step - loss: 0.5762 - acc: 1.0000
Epoch 51/80
100/100 [==============================] - 0s 70us/step - loss: 0.5669 - acc: 1.0000
Epoch 52/80
100/100 [==============================] - 0s 144us/step - loss: 0.5578 - acc: 1.0000
Epoch 53/80
100/100 [==============================] - 0s 58us/step - loss: 0.5466 - acc: 1.0000
Epoch 54/80
100/100 [==============================] - 0s 137us/step - loss: 0.5379 - acc: 1.0000
Epoch 55/80
100/100 [==============================] - 0s 58us/step - loss: 0.5314 - acc: 1.0000
Epoch 56/80
100/100 [==============================] - 0s 129us/step - loss: 0.5240 - acc: 1.0000
Epoch 57/80
100/100 [==============================] - 0s 63us/step - loss: 0.5131 - acc: 1.0000
Epoch 58/80
100/100 [==============================] - 0s 132us/step - loss: 0.5067 - acc: 1.0000
Epoch 59/80
100/100 [====

100/100 [==============================] - 0s 127us/step - loss: 0.8747 - acc: 1.0000
Epoch 33/80
100/100 [==============================] - 0s 67us/step - loss: 0.8542 - acc: 1.0000
Epoch 34/80
100/100 [==============================] - 0s 126us/step - loss: 0.8336 - acc: 1.0000
Epoch 35/80
100/100 [==============================] - 0s 60us/step - loss: 0.8110 - acc: 1.0000
Epoch 36/80
100/100 [==============================] - 0s 124us/step - loss: 0.7913 - acc: 1.0000
Epoch 37/80
100/100 [==============================] - 0s 72us/step - loss: 0.7722 - acc: 1.0000
Epoch 38/80
100/100 [==============================] - 0s 126us/step - loss: 0.7506 - acc: 1.0000
Epoch 39/80
100/100 [==============================] - 0s 56us/step - loss: 0.7342 - acc: 1.0000
Epoch 40/80
100/100 [==============================] - 0s 132us/step - loss: 0.7150 - acc: 1.0000
Epoch 41/80
100/100 [==============================] - ETA: 0s - loss: 0.7265 - acc: 1.000 - 0s 57us/step - loss: 0.6982 - acc: 1.0000

100/100 [==============================] - 0s 69us/step - loss: 1.4072 - acc: 0.9400
Epoch 16/80
100/100 [==============================] - 0s 118us/step - loss: 1.3734 - acc: 0.9500
Epoch 17/80
100/100 [==============================] - 0s 73us/step - loss: 1.3394 - acc: 0.9700
Epoch 18/80
100/100 [==============================] - 0s 111us/step - loss: 1.3119 - acc: 0.9600
Epoch 19/80
100/100 [==============================] - 0s 75us/step - loss: 1.2776 - acc: 0.9800
Epoch 20/80
100/100 [==============================] - 0s 118us/step - loss: 1.2407 - acc: 0.9900
Epoch 21/80
100/100 [==============================] - 0s 65us/step - loss: 1.2120 - acc: 0.9900
Epoch 22/80
100/100 [==============================] - 0s 117us/step - loss: 1.1809 - acc: 0.9900
Epoch 23/80
100/100 [==============================] - 0s 60us/step - loss: 1.1542 - acc: 0.9800
Epoch 24/80
100/100 [==============================] - 0s 135us/step - loss: 1.1212 - acc: 0.9800
Epoch 25/80
100/100 [================

...

100/100 [==============================] - 0s 67us/step - loss: 0.4927 - acc: 1.0000
Epoch 66/80
100/100 [==============================] - 0s 127us/step - loss: 0.4865 - acc: 1.0000
Epoch 67/80
100/100 [==============================] - 0s 59us/step - loss: 0.4786 - acc: 1.0000
Epoch 68/80
100/100 [==============================] - 0s 130us/step - loss: 0.4728 - acc: 1.0000
Epoch 69/80
100/100 [==============================] - 0s 64us/step - loss: 0.4641 - acc: 1.0000
Epoch 70/80
100/100 [==============================] - 0s 117us/step - loss: 0.4568 - acc: 1.0000
Epoch 71/80
100/100 [==============================] - 0s 69us/step - loss: 0.4519 - acc: 1.0000
Epoch 72/80
100/100 [==============================] - 0s 129us/step - loss: 0.4443 - acc: 1.0000
Epoch 73/80
100/100 [==============================] - 0s 54us/step - loss: 0.4395 - acc: 1.0000
Epoch 74/80
100/100 [==============================] - 0s 125us/step - loss: 0.4327 - acc: 1.0000
Epoch 75/80
100/100 [================

100/100 [==============================] - 0s 130us/step - loss: 0.6091 - acc: 1.0000
Epoch 49/80
100/100 [==============================] - 0s 57us/step - loss: 0.5980 - acc: 1.0000
Epoch 50/80
100/100 [==============================] - 0s 138us/step - loss: 0.5886 - acc: 1.0000
Epoch 51/80
100/100 [==============================] - 0s 60us/step - loss: 0.5760 - acc: 1.0000
Epoch 52/80
100/100 [==============================] - 0s 144us/step - loss: 0.5652 - acc: 1.0000
Epoch 53/80
100/100 [==============================] - 0s 69us/step - loss: 0.5552 - acc: 1.0000
Epoch 54/80
100/100 [==============================] - 0s 123us/step - loss: 0.5445 - acc: 1.0000
Epoch 55/80
100/100 [==============================] - 0s 60us/step - loss: 0.5354 - acc: 1.0000
Epoch 56/80
100/100 [==============================] - 0s 122us/step - loss: 0.5275 - acc: 1.0000
Epoch 57/80
100/100 [==============================] - 0s 56us/step - loss: 0.5197 - acc: 1.0000
Epoch 58/80
100/100 [================

Epoch 32/80
100/100 [==============================] - 0s 118us/step - loss: 0.8579 - acc: 1.0000
Epoch 33/80
100/100 [==============================] - 0s 66us/step - loss: 0.8305 - acc: 1.0000
Epoch 34/80
100/100 [==============================] - 0s 116us/step - loss: 0.8080 - acc: 1.0000
Epoch 35/80
100/100 [==============================] - 0s 74us/step - loss: 0.7873 - acc: 1.0000
Epoch 36/80
100/100 [==============================] - 0s 130us/step - loss: 0.7708 - acc: 1.0000
Epoch 37/80
100/100 [==============================] - 0s 74us/step - loss: 0.7477 - acc: 1.0000
Epoch 38/80
100/100 [==============================] - 0s 121us/step - loss: 0.7303 - acc: 1.0000
Epoch 39/80
100/100 [==============================] - 0s 61us/step - loss: 0.7119 - acc: 1.0000
Epoch 40/80
100/100 [==============================] - 0s 127us/step - loss: 0.6921 - acc: 1.0000
Epoch 41/80
100/100 [==============================] - 0s 57us/step - loss: 0.6793 - acc: 1.0000
Epoch 42/80
100/100 [====

100/100 [==============================] - 0s 69us/step - loss: 1.5596 - acc: 0.8800
Epoch 16/80
100/100 [==============================] - 0s 125us/step - loss: 1.5237 - acc: 0.8800
Epoch 17/80
100/100 [==============================] - 0s 63us/step - loss: 1.4827 - acc: 0.9000
Epoch 18/80
100/100 [==============================] - 0s 125us/step - loss: 1.4505 - acc: 0.9000
Epoch 19/80
100/100 [==============================] - 0s 61us/step - loss: 1.4124 - acc: 0.8900
Epoch 20/80
100/100 [==============================] - 0s 124us/step - loss: 1.3766 - acc: 0.9200
Epoch 21/80
100/100 [==============================] - 0s 57us/step - loss: 1.3342 - acc: 0.9300
Epoch 22/80
100/100 [==============================] - 0s 127us/step - loss: 1.3024 - acc: 0.9300
Epoch 23/80
100/100 [==============================] - 0s 59us/step - loss: 1.2656 - acc: 0.9200
Epoch 24/80
100/100 [==============================] - 0s 121us/step - loss: 1.2358 - acc: 0.9300
Epoch 25/80
100/100 [================

100/100 [==============================] - 0s 60us/step - loss: 2.0677 - acc: 0.5800
Epoch 19/20
100/100 [==============================] - 0s 77us/step - loss: 2.0324 - acc: 0.5800
Epoch 20/20
100/100 [==============================] - 0s 56us/step - loss: 1.9957 - acc: 0.6100
Epoch 1/80
100/100 [==============================] - 0s 71us/step - loss: 1.9618 - acc: 0.6500
Epoch 2/80
100/100 [==============================] - 0s 2ms/step - loss: 1.9186 - acc: 0.6800
Epoch 3/80
100/100 [==============================] - 0s 69us/step - loss: 1.8812 - acc: 0.6900
Epoch 4/80
100/100 [==============================] - 0s 127us/step - loss: 1.8415 - acc: 0.7500
Epoch 5/80
100/100 [==============================] - 0s 57us/step - loss: 1.8034 - acc: 0.7600
Epoch 6/80
100/100 [==============================] - 0s 127us/step - loss: 1.7647 - acc: 0.7700
Epoch 7/80
100/100 [==============================] - 0s 61us/step - loss: 1.7236 - acc: 0.8100
Epoch 8/80
100/100 [============================

In [45]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.9899
Score O2 avg: 0.9481


In [ ]:
# Model 0, with noise=10%

In [50]:
p=0.1
X_train_p = add_noise(X_train, p=p)
X_test_order1_p = add_noise(X_test_order1, p=p)
X_test_order2_p = add_noise(X_test_order2, p=p)
print(compute_diff(X_train_p, X_train))
print(compute_diff(X_test_order1_p, X_test_order1))
print(compute_diff(X_test_order2_p, X_test_order2))

0.0948333333333
0.09970625
0.100222916667


In [51]:
scores_o1, scores_o2 = model_trial(
    mlp0, X_train_p, Y_train, X_test_order1_p, X_test_order2_p, 
    nb_runs=10, nb_epoch=100, burn_period=20
)

Trial #1
Epoch 1/20
100/100 [==============================] - 1s 10ms/step - loss: 2.4340 - acc: 0.0600
Epoch 2/20
100/100 [==============================] - 0s 63us/step - loss: 2.3140 - acc: 0.0700
Epoch 3/20
100/100 [==============================] - 0s 71us/step - loss: 2.2614 - acc: 0.1300
Epoch 4/20
100/100 [==============================] - 0s 57us/step - loss: 2.2236 - acc: 0.1200
Epoch 5/20
100/100 [==============================] - 0s 60us/step - loss: 2.1889 - acc: 0.1500
Epoch 6/20
100/100 [==============================] - 0s 79us/step - loss: 2.1558 - acc: 0.1600
Epoch 7/20
100/100 [==============================] - 0s 69us/step - loss: 2.1280 - acc: 0.1700
Epoch 8/20
100/100 [==============================] - 0s 58us/step - loss: 2.0972 - acc: 0.2200
Epoch 9/20
100/100 [==============================] - 0s 58us/step - loss: 2.0647 - acc: 0.2700
Epoch 10/20
100/100 [==============================] - 0s 59us/step - loss: 2.0363 - acc: 0.2900
Epoch 11/20
100/100 [=========

100/100 [==============================] - 0s 72us/step - loss: 0.2784 - acc: 1.0000
Epoch 66/80
100/100 [==============================] - 0s 129us/step - loss: 0.2696 - acc: 1.0000
Epoch 67/80
100/100 [==============================] - 0s 70us/step - loss: 0.2573 - acc: 1.0000
Epoch 68/80
100/100 [==============================] - 0s 115us/step - loss: 0.2481 - acc: 1.0000
Epoch 69/80
100/100 [==============================] - 0s 72us/step - loss: 0.2389 - acc: 1.0000
Epoch 70/80
100/100 [==============================] - 0s 126us/step - loss: 0.2301 - acc: 1.0000
Epoch 71/80
100/100 [==============================] - 0s 54us/step - loss: 0.2228 - acc: 1.0000
Epoch 72/80
100/100 [==============================] - 0s 133us/step - loss: 0.2118 - acc: 1.0000
Epoch 73/80
100/100 [==============================] - 0s 58us/step - loss: 0.2049 - acc: 1.0000
Epoch 74/80
100/100 [==============================] - 0s 134us/step - loss: 0.1980 - acc: 1.0000
Epoch 75/80
100/100 [================

Epoch 49/80
100/100 [==============================] - 0s 76us/step - loss: 0.4860 - acc: 0.9900
Epoch 50/80
100/100 [==============================] - 0s 132us/step - loss: 0.4682 - acc: 0.9900
Epoch 51/80
100/100 [==============================] - 0s 77us/step - loss: 0.4536 - acc: 0.9900
Epoch 52/80
100/100 [==============================] - 0s 131us/step - loss: 0.4384 - acc: 0.9900
Epoch 53/80
100/100 [==============================] - 0s 65us/step - loss: 0.4269 - acc: 0.9900
Epoch 54/80
100/100 [==============================] - 0s 142us/step - loss: 0.4106 - acc: 0.9900
Epoch 55/80
100/100 [==============================] - 0s 70us/step - loss: 0.3979 - acc: 0.9900
Epoch 56/80
100/100 [==============================] - 0s 166us/step - loss: 0.3837 - acc: 0.9900
Epoch 57/80
100/100 [==============================] - 0s 71us/step - loss: 0.3662 - acc: 1.0000
Epoch 58/80
100/100 [==============================] - 0s 151us/step - loss: 0.3581 - acc: 1.0000
Epoch 59/80
100/100 [====

100/100 [==============================] - 0s 118us/step - loss: 0.6743 - acc: 0.9700
Epoch 33/80
100/100 [==============================] - 0s 77us/step - loss: 0.6522 - acc: 0.9700
Epoch 34/80
100/100 [==============================] - 0s 127us/step - loss: 0.6283 - acc: 0.9900
Epoch 35/80
100/100 [==============================] - 0s 72us/step - loss: 0.6048 - acc: 0.9900
Epoch 36/80
100/100 [==============================] - 0s 122us/step - loss: 0.5862 - acc: 1.0000
Epoch 37/80
100/100 [==============================] - 0s 66us/step - loss: 0.5651 - acc: 1.0000
Epoch 38/80
100/100 [==============================] - 0s 120us/step - loss: 0.5468 - acc: 1.0000
Epoch 39/80
100/100 [==============================] - 0s 67us/step - loss: 0.5291 - acc: 1.0000
Epoch 40/80
100/100 [==============================] - 0s 129us/step - loss: 0.5112 - acc: 1.0000
Epoch 41/80
100/100 [==============================] - 0s 72us/step - loss: 0.4901 - acc: 1.0000
Epoch 42/80
100/100 [================

100/100 [==============================] - 0s 66us/step - loss: 1.2372 - acc: 0.8600
Epoch 16/80
100/100 [==============================] - 0s 121us/step - loss: 1.2078 - acc: 0.8600
Epoch 17/80
100/100 [==============================] - 0s 72us/step - loss: 1.1751 - acc: 0.8600
Epoch 18/80
100/100 [==============================] - 0s 115us/step - loss: 1.1481 - acc: 0.8700
Epoch 19/80
100/100 [==============================] - 0s 76us/step - loss: 1.1162 - acc: 0.8800
Epoch 20/80
100/100 [==============================] - 0s 127us/step - loss: 1.0897 - acc: 0.8800
Epoch 21/80
100/100 [==============================] - 0s 59us/step - loss: 1.0605 - acc: 0.8800
Epoch 22/80
100/100 [==============================] - 0s 137us/step - loss: 1.0350 - acc: 0.8700
Epoch 23/80
100/100 [==============================] - 0s 57us/step - loss: 1.0069 - acc: 0.9000
Epoch 24/80
100/100 [==============================] - 0s 133us/step - loss: 0.9772 - acc: 0.9100
Epoch 25/80
100/100 [================

...

100/100 [==============================] - 0s 78us/step - loss: 0.3015 - acc: 1.0000
Epoch 66/80
100/100 [==============================] - 0s 118us/step - loss: 0.2927 - acc: 1.0000
Epoch 67/80
100/100 [==============================] - 0s 73us/step - loss: 0.2815 - acc: 1.0000
Epoch 68/80
100/100 [==============================] - 0s 115us/step - loss: 0.2703 - acc: 1.0000
Epoch 69/80
100/100 [==============================] - 0s 66us/step - loss: 0.2587 - acc: 1.0000
Epoch 70/80
100/100 [==============================] - 0s 136us/step - loss: 0.2523 - acc: 1.0000
Epoch 71/80
100/100 [==============================] - 0s 65us/step - loss: 0.2427 - acc: 1.0000
Epoch 72/80
100/100 [==============================] - 0s 122us/step - loss: 0.2335 - acc: 1.0000
Epoch 73/80
100/100 [==============================] - 0s 57us/step - loss: 0.2237 - acc: 1.0000
Epoch 74/80
100/100 [==============================] - 0s 135us/step - loss: 0.2161 - acc: 1.0000
Epoch 75/80
100/100 [================

100/100 [==============================] - 0s 135us/step - loss: 0.3891 - acc: 0.9800
Epoch 49/80
100/100 [==============================] - 0s 60us/step - loss: 0.3756 - acc: 0.9900
Epoch 50/80
100/100 [==============================] - 0s 139us/step - loss: 0.3602 - acc: 0.9900
Epoch 51/80
100/100 [==============================] - 0s 59us/step - loss: 0.3481 - acc: 0.9900
Epoch 52/80
100/100 [==============================] - 0s 142us/step - loss: 0.3379 - acc: 0.9800
Epoch 53/80
100/100 [==============================] - 0s 60us/step - loss: 0.3259 - acc: 0.9800
Epoch 54/80
100/100 [==============================] - 0s 121us/step - loss: 0.3163 - acc: 0.9900
Epoch 55/80
100/100 [==============================] - 0s 80us/step - loss: 0.3029 - acc: 0.9900
Epoch 56/80
100/100 [==============================] - 0s 127us/step - loss: 0.2914 - acc: 0.9900
Epoch 57/80
100/100 [==============================] - 0s 57us/step - loss: 0.2792 - acc: 0.9900
Epoch 58/80
100/100 [================

Epoch 32/80
100/100 [==============================] - 0s 129us/step - loss: 0.7899 - acc: 0.9700
Epoch 33/80
100/100 [==============================] - 0s 61us/step - loss: 0.7629 - acc: 0.9700
Epoch 34/80
100/100 [==============================] - 0s 131us/step - loss: 0.7377 - acc: 0.9900
Epoch 35/80
100/100 [==============================] - 0s 60us/step - loss: 0.7208 - acc: 1.0000
Epoch 36/80
100/100 [==============================] - 0s 128us/step - loss: 0.6933 - acc: 1.0000
Epoch 37/80
100/100 [==============================] - 0s 70us/step - loss: 0.6750 - acc: 1.0000
Epoch 38/80
100/100 [==============================] - 0s 123us/step - loss: 0.6536 - acc: 1.0000
Epoch 39/80
100/100 [==============================] - 0s 61us/step - loss: 0.6344 - acc: 1.0000
Epoch 40/80
100/100 [==============================] - 0s 121us/step - loss: 0.6104 - acc: 1.0000
Epoch 41/80
100/100 [==============================] - 0s 59us/step - loss: 0.5940 - acc: 1.0000
Epoch 42/80
100/100 [====

100/100 [==============================] - 0s 72us/step - loss: 1.1455 - acc: 0.8500
Epoch 16/80
100/100 [==============================] - 0s 130us/step - loss: 1.1138 - acc: 0.8600
Epoch 17/80
100/100 [==============================] - 0s 70us/step - loss: 1.0862 - acc: 0.8600
Epoch 18/80
100/100 [==============================] - 0s 127us/step - loss: 1.0610 - acc: 0.8500
Epoch 19/80
100/100 [==============================] - 0s 69us/step - loss: 1.0322 - acc: 0.8700
Epoch 20/80
100/100 [==============================] - 0s 144us/step - loss: 1.0052 - acc: 0.9100
Epoch 21/80
100/100 [==============================] - 0s 59us/step - loss: 0.9758 - acc: 0.9200
Epoch 22/80
100/100 [==============================] - 0s 128us/step - loss: 0.9498 - acc: 0.9300
Epoch 23/80
100/100 [==============================] - 0s 61us/step - loss: 0.9247 - acc: 0.9400
Epoch 24/80
100/100 [==============================] - 0s 119us/step - loss: 0.9034 - acc: 0.9400
Epoch 25/80
100/100 [================

100/100 [==============================] - 0s 59us/step - loss: 1.7968 - acc: 0.4700
Epoch 19/20
100/100 [==============================] - 0s 54us/step - loss: 1.7679 - acc: 0.5300
Epoch 20/20
100/100 [==============================] - 0s 71us/step - loss: 1.7359 - acc: 0.5600
Epoch 1/80
100/100 [==============================] - 0s 58us/step - loss: 1.7090 - acc: 0.5600
Epoch 2/80
100/100 [==============================] - 0s 2ms/step - loss: 1.6741 - acc: 0.6100
Epoch 3/80
100/100 [==============================] - 0s 67us/step - loss: 1.6432 - acc: 0.6200
Epoch 4/80
100/100 [==============================] - 0s 130us/step - loss: 1.6164 - acc: 0.6400
Epoch 5/80
100/100 [==============================] - 0s 74us/step - loss: 1.5868 - acc: 0.6300
Epoch 6/80
100/100 [==============================] - 0s 131us/step - loss: 1.5574 - acc: 0.6500
Epoch 7/80
100/100 [==============================] - 0s 53us/step - loss: 1.5273 - acc: 0.6500
Epoch 8/80
100/100 [============================

In [52]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.7156
Score O2 avg: 0.5908


In [53]:
scores_o1, scores_o2 = model_trial(
    mlp1, X_train_p, Y_train, X_test_order1_p, X_test_order2_p, 
    nb_runs=10, nb_epoch=100, burn_period=20
)

Trial #1
Epoch 1/20
100/100 [==============================] - 1s 12ms/step - loss: 2.8449 - acc: 0.1000
Epoch 2/20
100/100 [==============================] - 0s 65us/step - loss: 2.6751 - acc: 0.1300
Epoch 3/20
100/100 [==============================] - 0s 71us/step - loss: 2.6060 - acc: 0.1700
Epoch 4/20
100/100 [==============================] - 0s 74us/step - loss: 2.5453 - acc: 0.2100
Epoch 5/20
100/100 [==============================] - 0s 78us/step - loss: 2.4932 - acc: 0.2500
Epoch 6/20
100/100 [==============================] - 0s 70us/step - loss: 2.4568 - acc: 0.2600
Epoch 7/20
100/100 [==============================] - 0s 70us/step - loss: 2.4243 - acc: 0.3200
Epoch 8/20
100/100 [==============================] - 0s 63us/step - loss: 2.3863 - acc: 0.3100
Epoch 9/20
100/100 [==============================] - 0s 60us/step - loss: 2.3483 - acc: 0.3600
Epoch 10/20
100/100 [==============================] - 0s 58us/step - loss: 2.3131 - acc: 0.3400
Epoch 11/20
100/100 [=========

100/100 [==============================] - 0s 133us/step - loss: 0.6181 - acc: 1.0000
Epoch 65/80
100/100 [==============================] - 0s 65us/step - loss: 0.6072 - acc: 1.0000
Epoch 66/80
100/100 [==============================] - 0s 137us/step - loss: 0.5989 - acc: 1.0000
Epoch 67/80
100/100 [==============================] - 0s 60us/step - loss: 0.5901 - acc: 1.0000
Epoch 68/80
100/100 [==============================] - 0s 142us/step - loss: 0.5825 - acc: 1.0000
Epoch 69/80
100/100 [==============================] - 0s 69us/step - loss: 0.5761 - acc: 1.0000
Epoch 70/80
100/100 [==============================] - 0s 120us/step - loss: 0.5681 - acc: 1.0000
Epoch 71/80
100/100 [==============================] - 0s 71us/step - loss: 0.5614 - acc: 1.0000
Epoch 72/80
100/100 [==============================] - 0s 121us/step - loss: 0.5535 - acc: 1.0000
Epoch 73/80
100/100 [==============================] - 0s 67us/step - loss: 0.5461 - acc: 1.0000
Epoch 74/80
100/100 [================

Epoch 48/80
100/100 [==============================] - 0s 130us/step - loss: 0.7208 - acc: 1.0000
Epoch 49/80
100/100 [==============================] - 0s 67us/step - loss: 0.7087 - acc: 1.0000
Epoch 50/80
100/100 [==============================] - 0s 135us/step - loss: 0.6939 - acc: 1.0000
Epoch 51/80
100/100 [==============================] - 0s 62us/step - loss: 0.6823 - acc: 1.0000
Epoch 52/80
100/100 [==============================] - 0s 127us/step - loss: 0.6696 - acc: 1.0000
Epoch 53/80
100/100 [==============================] - ETA: 0s - loss: 0.6901 - acc: 1.000 - 0s 71us/step - loss: 0.6561 - acc: 1.0000
Epoch 54/80
100/100 [==============================] - 0s 123us/step - loss: 0.6461 - acc: 1.0000
Epoch 55/80
100/100 [==============================] - 0s 76us/step - loss: 0.6357 - acc: 1.0000
Epoch 56/80
100/100 [==============================] - 0s 121us/step - loss: 0.6275 - acc: 1.0000
Epoch 57/80
100/100 [==============================] - 0s 58us/step - loss: 0.6186 -

100/100 [==============================] - 0s 70us/step - loss: 1.2801 - acc: 0.9400
Epoch 32/80
100/100 [==============================] - 0s 128us/step - loss: 1.2560 - acc: 0.9600
Epoch 33/80
100/100 [==============================] - 0s 65us/step - loss: 1.2287 - acc: 0.9600
Epoch 34/80
100/100 [==============================] - 0s 132us/step - loss: 1.2061 - acc: 0.9500
Epoch 35/80
100/100 [==============================] - 0s 67us/step - loss: 1.1819 - acc: 0.9700
Epoch 36/80
100/100 [==============================] - 0s 119us/step - loss: 1.1613 - acc: 0.9700
Epoch 37/80
100/100 [==============================] - 0s 77us/step - loss: 1.1379 - acc: 0.9600
Epoch 38/80
100/100 [==============================] - 0s 131us/step - loss: 1.1177 - acc: 0.9800
Epoch 39/80
100/100 [==============================] - 0s 57us/step - loss: 1.0918 - acc: 0.9800
Epoch 40/80
100/100 [==============================] - 0s 130us/step - loss: 1.0697 - acc: 0.9800
Epoch 41/80
100/100 [================

Epoch 15/80
100/100 [==============================] - 0s 60us/step - loss: 1.6832 - acc: 0.7600
Epoch 16/80
100/100 [==============================] - 0s 136us/step - loss: 1.6536 - acc: 0.7800
Epoch 17/80
100/100 [==============================] - 0s 66us/step - loss: 1.6215 - acc: 0.8100
Epoch 18/80
100/100 [==============================] - 0s 131us/step - loss: 1.5948 - acc: 0.8100
Epoch 19/80
100/100 [==============================] - 0s 66us/step - loss: 1.5677 - acc: 0.8000
Epoch 20/80
100/100 [==============================] - 0s 123us/step - loss: 1.5393 - acc: 0.8100
Epoch 21/80
100/100 [==============================] - 0s 63us/step - loss: 1.5083 - acc: 0.8500
Epoch 22/80
100/100 [==============================] - 0s 136us/step - loss: 1.4791 - acc: 0.8700
Epoch 23/80
100/100 [==============================] - 0s 63us/step - loss: 1.4545 - acc: 0.8500
Epoch 24/80
100/100 [==============================] - 0s 123us/step - loss: 1.4272 - acc: 0.8600
Epoch 25/80
100/100 [====

...

100/100 [==============================] - 0s 67us/step - loss: 0.6544 - acc: 1.0000
Epoch 66/80
100/100 [==============================] - 0s 126us/step - loss: 0.6468 - acc: 0.9900
Epoch 67/80
100/100 [==============================] - 0s 65us/step - loss: 0.6348 - acc: 1.0000
Epoch 68/80
100/100 [==============================] - 0s 143us/step - loss: 0.6242 - acc: 1.0000
Epoch 69/80
100/100 [==============================] - 0s 70us/step - loss: 0.6137 - acc: 1.0000
Epoch 70/80
100/100 [==============================] - 0s 116us/step - loss: 0.6039 - acc: 1.0000
Epoch 71/80
100/100 [==============================] - 0s 81us/step - loss: 0.5946 - acc: 1.0000
Epoch 72/80
100/100 [==============================] - 0s 134us/step - loss: 0.5845 - acc: 1.0000
Epoch 73/80
100/100 [==============================] - 0s 62us/step - loss: 0.5778 - acc: 1.0000
Epoch 74/80
100/100 [==============================] - 0s 127us/step - loss: 0.5681 - acc: 1.0000
Epoch 75/80
100/100 [================

Epoch 49/80
100/100 [==============================] - 0s 62us/step - loss: 0.7814 - acc: 0.9900
Epoch 50/80
100/100 [==============================] - 0s 130us/step - loss: 0.7705 - acc: 0.9900
Epoch 51/80
100/100 [==============================] - 0s 65us/step - loss: 0.7560 - acc: 0.9900
Epoch 52/80
100/100 [==============================] - 0s 141us/step - loss: 0.7423 - acc: 0.9900
Epoch 53/80
100/100 [==============================] - 0s 64us/step - loss: 0.7300 - acc: 0.9900
Epoch 54/80
100/100 [==============================] - 0s 133us/step - loss: 0.7197 - acc: 0.9900
Epoch 55/80
100/100 [==============================] - 0s 61us/step - loss: 0.7097 - acc: 0.9900
Epoch 56/80
100/100 [==============================] - 0s 128us/step - loss: 0.6980 - acc: 0.9900
Epoch 57/80
100/100 [==============================] - 0s 67us/step - loss: 0.6865 - acc: 1.0000
Epoch 58/80
100/100 [==============================] - 0s 126us/step - loss: 0.6774 - acc: 1.0000
Epoch 59/80
100/100 [====

100/100 [==============================] - 0s 142us/step - loss: 1.2706 - acc: 0.8900
Epoch 33/80
100/100 [==============================] - 0s 61us/step - loss: 1.2496 - acc: 0.8900
Epoch 34/80
100/100 [==============================] - 0s 137us/step - loss: 1.2253 - acc: 0.9000
Epoch 35/80
100/100 [==============================] - 0s 60us/step - loss: 1.1997 - acc: 0.9100
Epoch 36/80
100/100 [==============================] - 0s 137us/step - loss: 1.1762 - acc: 0.9000
Epoch 37/80
100/100 [==============================] - 0s 70us/step - loss: 1.1560 - acc: 0.9100
Epoch 38/80
100/100 [==============================] - 0s 118us/step - loss: 1.1303 - acc: 0.9200
Epoch 39/80
100/100 [==============================] - 0s 78us/step - loss: 1.1080 - acc: 0.9500
Epoch 40/80
100/100 [==============================] - 0s 123us/step - loss: 1.0901 - acc: 0.9300
Epoch 41/80
100/100 [==============================] - 0s 59us/step - loss: 1.0670 - acc: 0.9400
Epoch 42/80
100/100 [================

100/100 [==============================] - 0s 82us/step - loss: 1.4800 - acc: 0.8600
Epoch 16/80
100/100 [==============================] - 0s 119us/step - loss: 1.4552 - acc: 0.8700
Epoch 17/80
100/100 [==============================] - 0s 77us/step - loss: 1.4337 - acc: 0.8500
Epoch 18/80
100/100 [==============================] - 0s 120us/step - loss: 1.4021 - acc: 0.9000
Epoch 19/80
100/100 [==============================] - 0s 85us/step - loss: 1.3746 - acc: 0.9000
Epoch 20/80
100/100 [==============================] - 0s 125us/step - loss: 1.3494 - acc: 0.9100
Epoch 21/80
100/100 [==============================] - 0s 67us/step - loss: 1.3231 - acc: 0.9000
Epoch 22/80
100/100 [==============================] - 0s 122us/step - loss: 1.2971 - acc: 0.9100
Epoch 23/80
100/100 [==============================] - 0s 61us/step - loss: 1.2727 - acc: 0.9300
Epoch 24/80
100/100 [==============================] - 0s 136us/step - loss: 1.2484 - acc: 0.9300
Epoch 25/80
100/100 [================

100/100 [==============================] - 0s 57us/step - loss: 2.1004 - acc: 0.5300
Epoch 19/20
100/100 [==============================] - 0s 77us/step - loss: 2.0717 - acc: 0.5600
Epoch 20/20
100/100 [==============================] - 0s 59us/step - loss: 2.0378 - acc: 0.5700
Epoch 1/80
100/100 [==============================] - 0s 57us/step - loss: 2.0039 - acc: 0.5900
Epoch 2/80
100/100 [==============================] - 0s 2ms/step - loss: 1.9759 - acc: 0.5800
Epoch 3/80
100/100 [==============================] - 0s 71us/step - loss: 1.9440 - acc: 0.6600
Epoch 4/80
100/100 [==============================] - 0s 133us/step - loss: 1.9097 - acc: 0.6900
Epoch 5/80
100/100 [==============================] - 0s 70us/step - loss: 1.8765 - acc: 0.6800
Epoch 6/80
100/100 [==============================] - 0s 124us/step - loss: 1.8478 - acc: 0.6900
Epoch 7/80
100/100 [==============================] - 0s 73us/step - loss: 1.8150 - acc: 0.6900
Epoch 8/80
100/100 [============================

In [54]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.7711
Score O2 avg: 0.6674
